In [9]:
from PIL import Image
from facenet_pytorch import InceptionResnetV1
from torchvision import transforms

import os
import numpy as np
import faiss
import torch

In [10]:
dataset_path = "Dataset"
os.listdir(dataset_path)   # Liet ke cac file va thu muc

['Avatar_Anil_Ramsook.jpg',
 'Avatar_Andrew_Cuomo.jpg',
 'Avatar_Camille_Lewis.jpg',
 'Avatar_Thang_Duong.jpg',
 'Avatar_Amy_Pascal.jpg',
 'Avatar_Andrew_Bernard.jpg',
 'Avatar_Tien_Huy.jpg',
 'Avatar_my-picture.jpg',
 'Avatar_Phuc_Thinh.JPG',
 'Avatar_Carla_Gay_Balingit.jpg',
 'Avatar_Dang_Nha.jpg',
 'Avatar_Amy_Redford.jpg',
 'Avatar_Hoang_Nguyen.jpg',
 'Avatar_Minh_Chau.jpg',
 'Avatar_Anh_Khoi.JPG',
 'Avatar_Aaron_Guiel.jpg',
 'Avatar_Thuan_Duong.jpg',
 'Avatar_Quoc_Thai.JPG',
 'Avatar_Aaron_Eckhart.jpg']

In [11]:
image_paths = []
labels = []

for filename in os.listdir(dataset_path):
    if filename.endswith(('jpg', 'JPG', 'jpeg', 'png')):
        image_paths.append(os.path.join(dataset_path, filename))
        fileNameWithoutTail = filename.split('.')[0]
        labels.append(fileNameWithoutTail[7:])

In [12]:
model = InceptionResnetV1(pretrained='vggface2').eval()   # Che do danh gia (khong phai che do training)

transform = transforms.Compose([
    transforms.Resize((300, 300)),   # Resize anh ve kich thuoc 300 x 300 pixel
    transforms.ToTensor(),   # Dua ve dang tensor (C, H, W) va chuan hoa ve doan [0, 1]
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])   # Dua ve phan phoi chuan trong doan [0, 1]
])

In [13]:
def extract_features(image_path, model):
    img = Image.open(image_path).convert('RGB')   # Ham convert() chuyen doi tat ca loai anh ve RGB
    image = transform(img).unsqueeze(0)

    with torch.no_grad():
        features = model(image)

    return features.squeeze().numpy()

In [14]:
index = faiss.IndexFlatL2(512)
label_map = []

for idx in range(len(image_paths)):
    features = extract_features(image_paths[idx], model)
    index.add(np.array([features]))
    label_map.append(labels[idx])

faiss.write_index(index, "improved_employee_images.index")
np.save("label_map.npy", label_map)

In [15]:
def search_query_image(query_image_path, k = 5):
    index = faiss.read_index("improved_employee_images.index")
    label_map = np.load("label_map.npy")

    query_vector = extract_features(query_image_path, model)

    distances, indices = index.search(np.array([query_vector]), k)

    results = []
    for idx in range(len(indices[0])):
        label = label_map[indices[0][idx]]
        distance = distances[0][idx]

        results.append((label, distance))

    return results

In [16]:
query_image_path = "test-picture.jpg"

print(search_query_image(query_image_path, 2))

[('my-picture', 0.16317278), ('Dang_Nha', 0.46942574)]
